## A demo using Hydrogen Hamiltonian with GPT-QE.

In [33]:
import torch
from gqe.mingpt.utils import set_seed

set_seed(3407)

In [34]:
from qwrapper.operator import PauliObservable
from gqe.mingpt.cost import EnergyCost
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.operator_pool.uccsd import UCCSD, do_generate_molecule
from gqe.common.initializer import HFStateInitializer
from gqe.util import get_device
from gqe.mingpt.callback import DefaultCallback, PrintMonitor, FileMonitor

# molecule = generate_molecule("Li", "H", 1.596, "sto-3g", bravyi_kitaev=False)
bond_length = 3.0
geometry = f"H 0.0 0.0 0.0\n" + f"Be 0.0 0.0 {bond_length}\n" + f"H 0.0 0.0 {2 * bond_length}\n"
molecule = do_generate_molecule(geometry, "sto-3g", bravyi_kitaev=False)
nqubit = 12

# prepare Hamiltonian
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)

# prepare operator_pool
uccsd = UCCSD(nqubit, molecule)
paulis = uccsd.paulis
paulis.append(PauliObservable("IIIIIIIIII"))
print('paulis', paulis)
num_operators = len(paulis)
initializer = HFStateInitializer(n_electrons=4)
pool = DefaultOperatorPool(paulis)
# time_pool = [1 / 320, -1 / 320, 1 / 160, -1 / 160, 1 / 80, -1 / 80, 1 / 40, -1 / 40, 0.05, -0.05, 0.1, -0.1, 0.2, -0.2]
time_pool = [1 / (2 ** j) for j in range(2, 12)]
time_pool.extend([-1 / (2 ** j) for j in range(2, 12)])
cost = EnergyCost(hamiltonian, initializer, pool, time_pool)


converged SCF energy = -15.0242100060364
paulis [+IIIXIIIIIIII, +IIIXZYIIIIII, +IIIXZZZYIIII, +IIIXZZZZZYII, +IIIXZZZZZZZY, +IIIYZXIIIIII, +IIIYZZZXIIII, +IIIYZZZZZXII, +IIIYZZZZZZZX, +IIXIIIIIIIII, +IIXXIIIIIIXY, +IIXXIIIIIIYX, +IIXXIIIIXYII, +IIXXIIIIYXII, +IIXXIIXYIIII, +IIXXIIYXIIII, +IIXXXYIIIIII, +IIXXYXIIIIII, +IIXYIIIIIIXX, +IIXYIIIIIIYY, +IIXYIIIIXXII, +IIXYIIIIYYII, +IIXYIIXXIIII, +IIXYIIYYIIII, +IIXYXXIIIIII, +IIXYYYIIIIII, +IIXZYIIIIIII, +IIXZZZYIIIII, +IIXZZZZZYIII, +IIXZZZZZZZYI, +IIYXIIIIIIXX, +IIYXIIIIIIYY, +IIYXIIIIXXII, +IIYXIIIIYYII, +IIYXIIXXIIII, +IIYXIIYYIIII, +IIYXXXIIIIII, +IIYXYYIIIIII, +IIYYIIIIIIXY, +IIYYIIIIIIYX, +IIYYIIIIXYII, +IIYYIIIIYXII, +IIYYIIXYIIII, +IIYYIIYXIIII, +IIYYXYIIIIII, +IIYYYXIIIIII, +IIYZXIIIIIII, +IIYZZZXIIIII, +IIYZZZZZXIII, +IIYZZZZZZZXI, +IXIIIIIIIIII, +IXXIIXZZZZYI, +IXXIIYZZZZXI, +IXXIXZZZZZZY, +IXXIYZZZZZZX, +IXYIIXZZZZXI, +IXYIIYZZZZYI, +IXYIXZZZZZZX, +IXYIYZZZZZZY, +IXZXIXZZZZZY, +IXZXIYZZZZZX, +IXZYIXZZZZZX, +IXZYIYZZZZZY, +IXZZZ

## FCI energy by diagonalization

In [42]:
from qwrapper.hamiltonian import compute_ground_state

print(compute_ground_state(hamiltonian))

-15.336459383306721


In [36]:
print("hf state:", hamiltonian.exact_value(initializer.init_circuit(12, [], "qulacs")))

hf state: -15.024210006036476


## Setup for GPT

In [37]:
# create a GPT instance
from gqe.mingpt.model import GPT

num_layers = 6



In [38]:
# create a Trainer object
from gqe.mingpt.trainer import Trainer
from gqe.mingpt.layer import LayerWiseTrainer, LayerWiseFactory

pool = DefaultOperatorPool(paulis)

time_pool = [1 / 320, -1 / 320, 1 / 160, -1 / 160, 1 / 80, -1 / 80, 1 / 40, -1 / 40, 0.05, -0.05, 0.1, -0.1, 0.2, -0.2]
# time_pool = [1 / (2 ** j) for j in range(layer, layer + 3)]
# time_pool.extend([-1 / (2 ** j) for j in range(layer, layer + 3)])

cost = EnergyCost(hamiltonian, initializer, pool, time_pool)


class CustomFactory(LayerWiseFactory):
    def generate_train_config(self, layer):
        train_config = Trainer.get_default_config()
        train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
        train_config.max_iters = 200
        train_config.num_workers = 5
        train_config.n_samples = 50
        return train_config

    def generate_model_config(self, layer):
        model_config = GPT.get_default_config()
        model_config.model_type = 'gpt2'
        model_config.vocab_size = self.generate_cost(layer).vocab_size()
        model_config.n_gates = 10  # The number of gates for each circuit
        model_config.block_size = model_config.n_gates
        model_config.temperature = 5  # Each gate is generated with probability exp(-temperature * logit)
        model_config.embd_pdrop = 0.1
        model_config.resid_pdrop = 0.1
        model_config.attn_pdrop = 0.1
        model_config.std = 0.02
        model_config.energy_offset = 14
        return model_config

    def generate_cost(self, layer):
        return cost


trainer = LayerWiseTrainer(CustomFactory(), num_layers, get_device())

In [39]:
print_monitors = []
file_monitors = []
for index in range(num_layers):
    file_monitor = FileMonitor()
    file_monitors.append(file_monitor)
    trainer.set_monitors(index, [PrintMonitor(), file_monitor])
trainer.run()
#torch.save(model.state_dict(), '../saved_models/gptqe_test_2')

layer: 1 starts running
number of parameters: 86.67M
running on device mps
iter_dt 0.00s; iter 0: train loss 0.23838 temperature: 5
mean_logits tensor([-15.2203, -14.8237, -15.0235, -15.0910, -14.7043, -15.0770, -15.3062,
        -14.9908, -15.0350, -15.0013, -14.9231, -14.9282, -14.8478, -14.8005,
        -15.0798, -14.8733, -14.9949, -14.7266, -15.3566, -14.9100, -14.9186,
        -14.8050, -15.0087, -14.9503, -14.8618, -14.8826, -15.1825, -15.0097,
        -14.9881, -14.8761, -15.1013, -14.9212, -15.2725, -15.1973, -15.0516,
        -14.7795, -15.0277, -14.8757, -14.9234, -14.9299, -14.7683, -14.8636,
        -15.0977, -15.1321, -15.1426, -14.9989, -14.7625, -14.8411, -14.7928,
        -15.2174], device='mps:0', grad_fn=<SubBackward0>)
energies: tensor([-14.8107, -14.9805, -15.0006, -15.0397, -14.9920, -15.0238, -15.1034,
        -14.9127, -15.0265, -14.9575, -15.0069, -14.9897, -14.9768, -14.9228,
        -14.9187, -14.9560, -14.9570, -15.0327, -14.9160, -14.9721, -15.0946,
       

In [40]:
indices = []
for fm in file_monitors:
    print(fm.min_energy)
    indices.extend(fm.min_indices)


-15.251511573791504
-15.273170471191406
-15.28294849395752
-15.290299415588379
-15.295649528503418
-15.298439025878906


In [41]:
from gqe.util import to_pqc
from gqe.vqa.initializer import HFVQAInitializer
from qwrapper.optimizer import AdamOptimizer
from qml.core.vqe import VQE
from qml.core.function import Energy

pqc = to_pqc(cost.sequence, indices)
initializer = HFVQAInitializer(4, nqubit, "qulacs")
optimizer = AdamOptimizer(maxiter=500)

energy = Energy(hamiltonian, nqubit, pqc)
vqe = VQE(energy, initializer, optimizer)
print(vqe.value())
vqe.exec()

print(vqe.value())
# cost.sequence.tool = "qiskit"
# print(file_monitor.min_indices)
# cost.sequence._get_circuit(file_monitor.min_indices).qc.draw(output="mpl", plot_barriers=True)

-15.298439051119118
0 -15.298439051119118
1 -15.299520514839363
2 -15.300419585689626
3 -15.301156913817392
4 -15.30175211383959
5 -15.302230724824113
6 -15.302622467295533
7 -15.302955854877544
8 -15.30325449538514
9 -15.303535478503822
10 -15.303809333160178
11 -15.304080885064355
12 -15.304350587127217
13 -15.30461613456676
14 -15.304874153178732
15 -15.305121592756834
16 -15.305356528414226
17 -15.305578304648755
18 -15.305787254530067
19 -15.305984301301581
20 -15.30617060430582
21 -15.306347317588168
22 -15.306515499940485
23 -15.306676131478627
24 -15.30683013295197
25 -15.306978324741863
26 -15.307121349901756
27 -15.30725963889212
28 -15.307393449526023
29 -15.30752293603594
30 -15.307648188528226
31 -15.307769235676226
32 -15.307886048464807
33 -15.307998579430773
34 -15.308106838233446
35 -15.3082109743121
36 -15.308311324753875
37 -15.30840839124755
38 -15.308502747030156
39 -15.308594924914303
40 -15.308685348117722
41 -15.308774320724622
42 -15.308862046561245
43 -15.3089